# 文本分类处理流程

贝叶斯新闻分类任务

- 新闻数据集处理

爬取的新闻数据，需要我们对文本数据进行很多预处理才能使用

- 文本分词

通常我们处理的都是词而不是一篇文章

- 去停用词

停用词会对结果产生不好的影响，所以一定得把他们去剔除掉

- 构建文本特征

如何构建合适特征是自然语言处理中最重要的一步，这俩我们选择两种方案来进行对比

- 贝叶斯分类

基于贝叶斯算法来完成最终的分类任务

# 贝叶斯新闻分类

数据源：http://www.sogou.com/labs/resource/ca.php

## 读取数据

In [35]:
#读取数据
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import jieba
df_news = pd.read_table('newsdata.txt',names=['category','theme','URL','content'],encoding='utf-8')
df_news = df_news.dropna()  #删除缺失值
df_news.head()

,category,theme,URL,content
0,汽车,新辉腾 4.2 V8 4座加长Individual版2011款 最新报价,http://auto.data.people.com.cn/model_15782/,经销商 电话 试驾/订车U憬杭州滨江区江陵路1780号4008-112233转5864#保常...
1,汽车,918 Spyder概念车,http://auto.data.people.com.cn/prdview_165423....,呼叫热线 4008-100-300 服务邮箱 kf@peopledaily.com.cn
2,汽车,日内瓦亮相 MINI性能版/概念车-1.6T引擎,http://auto.data.people.com.cn/news/story_5249...,"MINI品牌在二月曾经公布了最新的MINI新概念车Clubvan效果图,不过现在在日内瓦车展..."
3,汽车,清仓大甩卖一汽夏利N5威志V2低至3.39万,http://auto.data.people.com.cn/news/story_6144...,"清仓大甩卖!一汽夏利N5､威志V2低至3.39万=日,启新中国一汽强势推出一汽夏利N5､威志..."
4,汽车,大众敞篷家族新成员 高尔夫敞篷版实拍,http://auto.data.people.com.cn/news/story_5686...,"在今年3月的日内瓦车展上,我们见到了高尔夫家族的新成员,高尔夫敞篷版,这款全新敞篷车受到了众..."


In [39]:
df_news['content']=df_news['content'].str.replace('\d+', '',regex=True) #替换

In [41]:
df_news['content']

0       经销商 电话 试驾/订车U憬杭州滨江区江陵路号-转#保常叮万阒菔邪自魄白云大道北号;广州市天...
1                      呼叫热线 -- 服务邮箱 kf@peopledaily.com.cn
2       MINI品牌在二月曾经公布了最新的MINI新概念车Clubvan效果图,不过现在在日内瓦车展...
3       清仓大甩卖!一汽夏利N､威志V低至.万=日,启新中国一汽强势推出一汽夏利N､威志V＂清仓大甩...
4       在今年月的日内瓦车展上,我们见到了高尔夫家族的新成员,高尔夫敞篷版,这款全新敞篷车受到了众多...
                              ...                        
4995    随着天气逐渐炎热,补水变得日益重要｡据美国《跑步世界》杂志报道,喝水并不是为身体补充水分的唯...
4996    我其实不想说这些话刺激他,他也是不得已｡可是,我又该怎样说,怎样做?我只能走,离开这个伤心地...
4997    岁刘晓庆最新嫩照O衷诘牧跸庆绝对看不出她已经岁了,她绝对可以秒杀刘亦菲､范冰冰这类美女｡无论...
4998    导语:做了爸爸就是一种幸福,无论是领养还是亲生,更何况出现在影视剧中｡时尚圈永远是需要领军人...
4999    全球最美女人合成图::国整形外科教授李承哲,在国际学术杂志美容整形外科学会学报发表了考虑种族...
Name: content, Length: 5000, dtype: object

In [47]:
df_news['content']=df_news['content'].str.replace('[A-z]', '',regex=True) 

In [26]:
df_news.shape

(5000, 4)

In [48]:
content = df_news.content.values.tolist() #将每一篇文章转换成一个list
content[0]

'经销商 电话 试驾/订车憬杭州滨江区江陵路号-转#保常叮万阒菔邪自魄白云大道北号;广州市天河区黄埔大道西号富力盈泰大厦室-转#保常福万蕉省淄博市张店区山泉路号-转#保常叮万罅保税区黄海西三路号-转#保玻埃万粕绞新纺锨复兴路号-转#保常叮万泄云南昆明市度假区滇池路号-转#保常叮万川市兴庆区丽景北街号-转#保常叮万尔滨市道外区先锋路号-转#保矗福万ど呈刑煨那桂花坪街道雀园路口/星沙中南汽车世界区号-转#保常梗万浜菏信塘城经济开发区盘龙汽车城#矗埃埃福转#保常叮万阒莘禺区市广路号(祈福食街旁)#矗埃埃福转#保常叮万侄新区御桥路号-转#保常福万不帐『戏适邪河工业区纬一路号.万虾斜ι角江杨南路号-转#保常叮万奚蕉路号-转#保常叮万本┦谐阳区北四环望京街号-转#保玻福万本┦胁平区立汤路亚北博晟汽车汇展中心#保埃福万=西省南昌市青山湖区科技大道号.万苄耸信劢工业区康宁路车管所对面#保常叮万暇┦薪宁区天元中路号-转#保常叮万ご菏形餍戮济技术开发区长沈路号.万家庄市北二环东路号河北国际汽车贸易园区#矗埃埃福转#保矗福万壅⑶城港路号广达车城永兴路号.万蜓羰刑西区北二中路号-转#保常叮万啥际星嘌虼蟮溃保福负牛ㄐ挛幕宫对面)#保矗常万赡省沈阳市皇姑区鸭绿江街号甲(长客总站北行米)#保矗福万钲谑新藓区罗芳立交六星汽车园进口大众店-转#保担埃万ご憾环城路号.万'

## 中文分词和去停用词

In [49]:
stopwords=pd.read_csv('停用词.txt',header=None)
stopwords.head()

,0
0,$
1,0
2,1
3,2
4,3


In [50]:
stopwords =  stopwords[0].tolist()
stopwords.append('\r\n')

In [51]:
stopwords

['$',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '?',
 '_',
 '“',
 '”',
 '、',
 '。',
 '《',
 '》',
 '一',
 '一些',
 '一何',
 '一切',
 '一则',
 '一方面',
 '一旦',
 '一来',
 '一样',
 '一般',
 '一转眼',
 '万一',
 '上',
 '上下',
 '下',
 '不',
 '不仅',
 '不但',
 '不光',
 '不单',
 '不只',
 '不外乎',
 '不如',
 '不妨',
 '不尽',
 '不尽然',
 '不得',
 '不怕',
 '不惟',
 '不成',
 '不拘',
 '不料',
 '不是',
 '不比',
 '不然',
 '不特',
 '不独',
 '不管',
 '不至于',
 '不若',
 '不论',
 '不过',
 '不问',
 '与',
 '与其',
 '与其说',
 '与否',
 '与此同时',
 '且',
 '且不说',
 '且说',
 '两者',
 '个',
 '个别',
 '临',
 '为',
 '为了',
 '为什么',
 '为何',
 '为止',
 '为此',
 '为着',
 '乃',
 '乃至',
 '乃至于',
 '么',
 '之',
 '之一',
 '之所以',
 '之类',
 '乌乎',
 '乎',
 '乘',
 '也',
 '也好',
 '也罢',
 '了',
 '二来',
 '于',
 '于是',
 '于是乎',
 '云云',
 '云尔',
 '些',
 '亦',
 '人',
 '人们',
 '人家',
 '什么',
 '什么样',
 '今',
 '介于',
 '仍',
 '仍旧',
 '从',
 '从此',
 '从而',
 '他',
 '他人',
 '他们',
 '以',
 '以上',
 '以为',
 '以便',
 '以免',
 '以及',
 '以故',
 '以期',
 '以来',
 '以至',
 '以至于',
 '以致',
 '们',
 '任',
 '任何',
 '任凭',
 '似的',
 '但',
 '但凡',
 '但是',
 '何',
 '何以',
 '何况',
 '何处',
 '何时',
 '余外',
 '作为',
 '你',
 '你们'

In [52]:
content[0]

'经销商 电话 试驾/订车憬杭州滨江区江陵路号-转#保常叮万阒菔邪自魄白云大道北号;广州市天河区黄埔大道西号富力盈泰大厦室-转#保常福万蕉省淄博市张店区山泉路号-转#保常叮万罅保税区黄海西三路号-转#保玻埃万粕绞新纺锨复兴路号-转#保常叮万泄云南昆明市度假区滇池路号-转#保常叮万川市兴庆区丽景北街号-转#保常叮万尔滨市道外区先锋路号-转#保矗福万ど呈刑煨那桂花坪街道雀园路口/星沙中南汽车世界区号-转#保常梗万浜菏信塘城经济开发区盘龙汽车城#矗埃埃福转#保常叮万阒莘禺区市广路号(祈福食街旁)#矗埃埃福转#保常叮万侄新区御桥路号-转#保常福万不帐『戏适邪河工业区纬一路号.万虾斜ι角江杨南路号-转#保常叮万奚蕉路号-转#保常叮万本┦谐阳区北四环望京街号-转#保玻福万本┦胁平区立汤路亚北博晟汽车汇展中心#保埃福万=西省南昌市青山湖区科技大道号.万苄耸信劢工业区康宁路车管所对面#保常叮万暇┦薪宁区天元中路号-转#保常叮万ご菏形餍戮济技术开发区长沈路号.万家庄市北二环东路号河北国际汽车贸易园区#矗埃埃福转#保矗福万壅⑶城港路号广达车城永兴路号.万蜓羰刑西区北二中路号-转#保常叮万啥际星嘌虼蟮溃保福负牛ㄐ挛幕宫对面)#保矗常万赡省沈阳市皇姑区鸭绿江街号甲(长客总站北行米)#保矗福万钲谑新藓区罗芳立交六星汽车园进口大众店-转#保担埃万ご憾环城路号.万'

In [53]:
content_S = []
for line in content:
    current_segment = jieba.lcut(line) #对每一篇文章进行分词，返回的列表
    current_segment=[segment for segment in current_segment if segment not in stopwords] #去停用词后的列表
    segments=" ".join(current_segment)
    content_S.append(segments) #保存分词的结果

In [54]:
content_S

['经销商   电话   试驾 / 订车 憬 杭州 滨江区 江陵 路 号 - 转 # 保常 叮 万 阒 菔 邪 自魄 白云 大道北 号 ; 广州市 天河区 黄埔 大道 西号 富力 盈泰 大厦 室 - 转 # 保常福 万蕉省 淄博市 张店区 山泉 路 号 - 转 # 保常 叮 万 罅 保税区 黄海 西 三路 号 - 转 # 保玻埃万粕 绞 新 纺锨 复兴路 号 - 转 # 保常 叮万泄 云南 昆明市 度假区 滇池 路 号 - 转 # 保常 叮 万川 市 兴庆区 丽景 北街 号 - 转 # 保常 叮万尔滨市 道外区 先锋 路 号 - 转 # 保 矗福万 ど 呈刑 煨 桂花 坪 街道 雀园 路口 / 星沙 中南 汽车 世界 区 号 - 转 # 保常 梗 万 浜 菏 信塘城 经济 开发区 盘龙 汽车城 # 矗埃埃福转 # 保常 叮 万 阒 莘禺 区市 广路 号 ( 祈福 食街 旁 ) # 矗埃埃福转 # 保常 叮万侄 新区 御桥 路 号 - 转 # 保常福 万 帐 『 戏适 邪河 工业区 纬 一路 号 . 万虾 斜 ι 角江 杨 南路 号 - 转 # 保常 叮万 奚蕉路 号 - 转 # 保常 叮万本 ┦ 谐阳区 北四环 望京 街号 - 转 # 保玻福 万本 ┦ 胁 平区立 汤路 亚北博晟 汽车 汇展 中心 # 保埃福 万 = 西省 南昌市 青山湖区 科技 大道 号 . 万苄 耸信 劢 工业区 康宁 路 车管所 对面 # 保常 叮万暇 ┦ 薪宁区 天元 中路 号 - 转 # 保常 叮万 ご 菏形 餍 戮济 技术开发区 长沈路 号 . 万家 庄市 北二环 东路 号 河北 国际 汽车贸易 园区 # 矗埃埃福转 # 保 矗福万 壅 ⑶ 城港路 号 广达 车城 永兴路 号 . 万蜓 羰刑 西区 北二 中路 号 - 转 # 保常 叮万 啥际星 嘌 虼 蟮 溃 保福 负牛 ㄐ 挛幕宫 对面 ) # 保 矗常万 赡省 沈阳市 皇姑区 鸭绿江 街号 甲 ( 长客 总站 北行 米 ) # 保 矗福 万钲谑 新藓区 罗芳 立交 六星 汽车 园 进口 大众 店 - 转 # 保担 埃万 ご 憾 环城路 号 . 万',
 '呼叫 热线   --   服务 邮箱   @ ..',
 '品牌 二月 曾经 公布 最新 新 概念车 效果图 , 现在 日内瓦 车展 , 品牌 带来 全新   版 概念车 更 性能

In [55]:
df=pd.DataFrame({'contents_clean':content_S,'label':df_news['category']})
df.head()#去掉停用词后

,contents_clean,label
0,经销商 电话 试驾 / 订车 憬 杭州 滨江区 江陵 路 号 - 转 # 保常 叮 ...,汽车
1,呼叫 热线 -- 服务 邮箱 @ ..,汽车
2,"品牌 二月 曾经 公布 最新 新 概念车 效果图 , 现在 日内瓦 车展 , 品牌 带来 全...",汽车
3,"清仓 甩卖 ! 一汽 夏利 ､ 威志低 . 万 = 日 , 启新 中国 一汽 强势 推出 一...",汽车
4,"今年 月 日内瓦 车展 , 见到 高尔夫 家族 新 成员 , 高尔夫 敞篷版 , 款 全新 ...",汽车


## 提取标签

In [56]:
label_mapping = {"时尚": 0, "汽车": 1, "财经": 2, "科技": 3, "健康": 4, "体育":5, "教育": 6,"文化": 7,"军事": 8,"娱乐": 9}
df['label'] = df['label'].map(label_mapping) #构建一个映射方法
df.head()

,contents_clean,label
0,经销商 电话 试驾 / 订车 憬 杭州 滨江区 江陵 路 号 - 转 # 保常 叮 ...,1
1,呼叫 热线 -- 服务 邮箱 @ ..,1
2,"品牌 二月 曾经 公布 最新 新 概念车 效果图 , 现在 日内瓦 车展 , 品牌 带来 全...",1
3,"清仓 甩卖 ! 一汽 夏利 ､ 威志低 . 万 = 日 , 启新 中国 一汽 强势 推出 一...",1
4,"今年 月 日内瓦 车展 , 见到 高尔夫 家族 新 成员 , 高尔夫 敞篷版 , 款 全新 ...",1


## 训练集和测试集划分

In [57]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['contents_clean'].values, df['label'].values, random_state=1)

## 提取特征，训练模型，评估模型

### 词袋模型

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
train_feature = vec.fit_transform(x_train) #训练集转换为向量
train_feature

<3750x87289 sparse matrix of type '<class 'numpy.int64'>'
	with 461523 stored elements in Compressed Sparse Row format>

In [59]:
y_train

array([7, 2, 2, ..., 2, 8, 1], dtype=int64)

In [60]:
test_feature = vec.transform(x_test) #测试集转换为向量
test_feature.shape

(1250, 87289)

In [61]:
from sklearn.naive_bayes import MultinomialNB #贝叶斯模型,多项式
classifier = MultinomialNB() 
classifier.fit(train_feature, y_train) #训练
classifier.score(test_feature, y_test) #准确率

0.824

### TF-IDF模型

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)
train_feature=vectorizer.transform(x_train)

test_feature = vectorizer.transform(x_test)

classifier = MultinomialNB()
classifier.fit(train_feature, y_train)
classifier.score(test_feature, y_test)

0.828

In [24]:
train_feature.shape

(3750, 85751)

## 网格搜索参数优化

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=2000)
vectorizer.fit(x_train)
train_feature=vectorizer.transform(x_train)

test_feature = vectorizer.transform(x_test)

classifier = MultinomialNB()
classifier.fit(train_feature, y_train)
classifier.score(test_feature, y_test)

0.7984

In [75]:
classifier

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [63]:
from sklearn.model_selection import GridSearchCV
params={'alpha':[0.1,0.2,0.3,0.4]}
classifier = MultinomialNB()
grid_search=GridSearchCV(classifier,param_grid=params,cv=10)
grid_search.fit(train_feature, y_train)
grid_search.best_params_

{'alpha': 0.1}

In [64]:
grid_search

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': [0.1, 0.2, 0.3, 0.4]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

### 模型评估

#### 模型得分

In [66]:
grid_search.score(test_feature, y_test)#模型得分,准确率

0.8304

#### 分类报告

In [67]:
label_mapping = {"汽车": 1, "财经": 2, "科技": 3, "健康": 4, "体育":5, "教育": 6,"文化": 7,"军事": 8,"娱乐": 9,"时尚": 0}

In [68]:
from sklearn.metrics import classification_report
pred_test=grid_search.predict(test_feature) #预测结果
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.84      0.61      0.70       122
           1       0.97      0.98      0.98       124
           2       0.84      0.88      0.86       121
           3       0.89      0.86      0.88       127
           4       0.82      0.84      0.83       150
           5       0.88      0.98      0.93       108
           6       0.90      0.83      0.86       129
           7       0.74      0.54      0.62       111
           8       0.88      0.88      0.88       130
           9       0.62      0.88      0.72       128

   micro avg       0.83      0.83      0.83      1250
   macro avg       0.84      0.83      0.83      1250
weighted avg       0.84      0.83      0.83      1250



#### 混淆矩阵

In [69]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test, pred_test))#混淆矩阵

[[ 74   1   0   4  15   4   0   3   0  21]
 [  0 122   0   0   0   2   0   0   0   0]
 [  0   0 107   6   4   1   0   1   1   1]
 [  1   1   5 109   0   1   4   0   2   4]
 [  9   0   4   1 126   0   6   1   1   2]
 [  0   0   0   0   0 106   0   0   2   0]
 [  3   0   2   0   5   0 107   3   5   4]
 [  0   0   4   1   3   1   1  60   4  37]
 [  0   2   5   1   1   4   0   2 115   0]
 [  1   0   1   0   0   2   1  11   0 112]]


## 逻辑回归模型

In [70]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(train_feature, y_train)
classifier.score(test_feature, y_test)

0.8376

In [ ]:
#参数优化
from sklearn.model_selection import GridSearchCV
params={'C':[1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8]}
classifier = LogisticRegression()
grid_search=GridSearchCV(classifier,param_grid=params,cv=10)
grid_search.fit(train_feature, y_train)
grid_search.best_params_

In [ ]:
grid_search.score(test_feature, y_test)

## 决策树模型

In [ ]:
from sklearn import tree
classifier = tree.DecisionTreeClassifier()
classifier.fit(train_feature, y_train)
classifier.score(test_feature, y_test)

In [ ]:
#参数优化
from sklearn.model_selection import GridSearchCV
params={'max_depth':[130,140,150,160,170]}
classifier = tree.DecisionTreeClassifier()
grid_search=GridSearchCV(classifier,param_grid=params,cv=10)
grid_search.fit(train_feature, y_train)
grid_search.best_params_

In [ ]:
grid_search.score(test_feature, y_test)

## 神经网络

In [98]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
iris = load_iris()
X=iris.data
y=iris.target
clf = MLPClassifier(hidden_layer_sizes=(200,200))
clf.fit(X, y)
clf.score(X, y)

0.98

In [99]:
train_feature.shape

(3750, 2000)

In [100]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(hidden_layer_sizes=(4000))
classifier.fit(train_feature, y_train)
classifier.score(test_feature, y_test)

0.7432